In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from helper_functions import Helper
from typing import Dict

In [2]:
def cleanPanel(df: pd.DataFrame, asset_universe_dict: Dict[str, list]) -> tuple:
    """ A whole lotta code that should be a bunch of separate functions to
        clean the panel down to columnns with full data and then a macro
        dataframe for the columns that had too many missing for the cross-
        sectional dataframe.
    """
    # Clean asset column
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)
    df = df.rename(columns={'asset': 'asset_cm'})
    assert 0 == df.asset_cm.isnull().sum()
    assert len(df) == df[df.asset_cm.isin(asset_universe)].shape[0]

    # Convert date column to datetime format and remove timezone information
    assert 0 == df.date.isnull().sum()
    assert len(df) == df[df.date.dt.minute==0].shape[0]
    df = df.sort_values(by=['date', 'asset_cm'])

    # Cut down to relevant dates between July 1, 2016 and January 2, 2023
    df = df[(df['date'] >= '2016-07-01') & (df['date'] <= '2023-01-02')]

    # Drop any duplicates
    df = df.drop_duplicates(subset=['date', 'asset_cm'])

    # Convert all columns to float32
    cols = list(df.columns.values)
    cols.remove('date')
    cols.remove('asset_cm')
    for col in cols:
        df[col] = df[col].astype('float32')

    # Set column order
    cols.sort()
    df = df[['date', 'asset_cm']+cols]

    # Determine the first tradable date for each asset
    first_tradable_dates = {}
    for date, assets in asset_universe_dict.items():
        for asset in assets:
            if asset not in first_tradable_dates:
                first_tradable_dates[asset] = date
    first_tradable_df = pd.DataFrame(list(first_tradable_dates.items()), columns=["asset_cm", "first_tradable_date"])
    first_tradable_df['first_tradable_date'] = pd.to_datetime(first_tradable_df['first_tradable_date'], 
                                                            utc=True).dt.tz_localize(None)

    # Shift first tradable date back three months to form lagged covariates
    first_tradable_df['first_tradable_date'] = first_tradable_df.first_tradable_date - pd.DateOffset(months=3)

    # For each asset in my panel, drop any rows that are before the first tradable date
    for asset in df.asset_cm.unique():
        first_tradable_date = first_tradable_df[first_tradable_df.asset_cm==asset].first_tradable_date.values[0]
        df = df[~((df.asset_cm==asset) & (df.date<first_tradable_date))]

    # Loop over all assets to add any missing datetimes and fill missing columns
    final_df = pd.DataFrame()
    df = df.sort_values(by=['date', 'asset_cm'], ignore_index=True)
    assets = list(np.unique(df.asset_cm.values))
    for asset in assets:
        # subset to asset of interest
        asset_df = df[df.asset_cm==asset].copy()

        # determine the date gaps
        date_gaps = []
        dates = asset_df.date.values
        for i in range(1, len(dates)):
            date_gaps.append(np.timedelta64(dates[i]-dates[i-1], 'h').astype(int))
        
        # determine new datetimes to add
        indices_to_expand = [i for i in range(len(date_gaps)) if (date_gaps[i] > 1) & (date_gaps[i] < 32*24)]
        num_datetime_to_add = [date_gaps[i] for i in range(len(date_gaps)) if (date_gaps[i] > 1) & (date_gaps[i] < 32*24)]
        start_datetimes = dates[indices_to_expand]
        new_datetimes = []
        for i in range(len(start_datetimes)):
            start_datetime = start_datetimes[i]
            datetime_to_add = num_datetime_to_add[i]
            for j in range(1, datetime_to_add):
                new_datetimes.append(start_datetime+np.timedelta64(j, 'h'))

        # add the new datetimes to the asset df
        new_asset_df = pd.DataFrame(data={'date': new_datetimes})
        new_asset_df['asset_cm'] = asset
        asset_df = pd.concat((asset_df, new_asset_df))
        asset_df = asset_df.sort_values(by='date', ignore_index=True)

        # drop any duplicates added
        asset_df = asset_df.drop_duplicates(subset=['date'])

        # forward fill 
        asset_df = asset_df.ffill(limit=32*24)

        # if asset contains nonmissing obs in a column but starts missing, then fill first obs until nonmissing with 0
        cols_to_fill_first_row_with_zero = list(asset_df.columns[asset_df.notna().any() & asset_df.iloc[0].isna()].values)
        for col in cols_to_fill_first_row_with_zero:
            first_non_missing_index = asset_df[col].first_valid_index()
            asset_df.loc[:(first_non_missing_index-1), col] = 0

        # add data to master df
        final_df = pd.concat((final_df, asset_df))

    # reset df name
    del df
    df = final_df.copy()

    # Reset column names to snake case
    def insert_underscore_before_numbers_and_keywords(string_list):
        modified_list = []
        for s in string_list:
            s = re.sub(r'(?<=[^\d_])(\d)', r'_\1', s, count=1)  # Insert underscore before the first number
            s = re.sub(r'(?<![A-Za-z_])(usd|cnt)', r'_\1', s, flags=re.IGNORECASE)  # Insert underscore before "usd" or "cnt"
            modified_list.append(s)
        return modified_list

    cols = list(df.columns.values)
    cols = cols[2:-7] # assumes a particular ordering

    new_col_names = [re.sub(r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', '_', s).lower() for s in cols]
    new_col_names = insert_underscore_before_numbers_and_keywords(new_col_names)

    rename_dict = {old_col: new_col for old_col, new_col in zip(cols, new_col_names)}
    df = df.rename(columns=rename_dict)

    # Ensure all columns have cm in name
    df.columns = [col if col == 'date' or col.endswith('_cm') else col + '_cm' for col in df.columns]

    # Clean missing in bid ask and form bid ask difference
    df['usd_bid_size_cm'] = df.groupby('asset_cm')['usd_bid_size_cm'].fillna(method='ffill')
    df['usd_ask_size_cm'] = df.groupby('asset_cm')['usd_ask_size_cm'].fillna(method='ffill')
    df['usd_bid_size_cm'].fillna(df.groupby('date')['usd_bid_size_cm'].transform('median'), inplace=True)
    df['usd_ask_size_cm'].fillna(df.groupby('date')['usd_ask_size_cm'].transform('median'), inplace=True)
    assert 0 == df.usd_bid_size_cm.isnull().sum()
    assert 0 == df.usd_ask_size_cm.isnull().sum()
    df['usd_bid_cm'] = df.groupby('asset_cm')['usd_bid_cm'].fillna(method='ffill')
    df['usd_ask_cm'] = df.groupby('asset_cm')['usd_ask_cm'].fillna(method='ffill')
    df['bidask_cm'] = df.usd_ask_cm-df.usd_bid_cm
    df['bidask_cm'].fillna(df.groupby('date')['bidask_cm'].transform('median'), inplace=True)
    assert 0 == df.bidask_cm.isnull().sum()
    df.loc[df.usd_bid_cm.isnull(), 'usd_bid_cm'] = df.usd_per_token_cm - df.bidask_cm/2
    df.loc[df.usd_ask_cm.isnull(), 'usd_ask_cm'] = df.usd_per_token_cm + df.bidask_cm/2
    assert 0 == df.usd_bid_cm.isnull().sum()
    assert 0 == df.usd_ask_cm.isnull().sum()

    # Clean price
    df['usd_per_token_cm'] = df.groupby('asset_cm')['usd_per_token_cm'].fillna(method='ffill')

    # Fix outliers in bid ask columns
    usd_bid_cm = df['usd_bid_cm'].values
    usd_per_token_cm = df['usd_per_token_cm'].values
    bidask_cm = df['bidask_cm'].values
    mask = usd_bid_cm > usd_per_token_cm
    usd_bid_cm[mask] = usd_per_token_cm[mask] - bidask_cm[mask]/2
    df['usd_bid_cm'] = usd_bid_cm
    usd_ask_cm = df['usd_ask_cm'].values
    usd_per_token_cm = df['usd_per_token_cm'].values
    bidask_cm = df['bidask_cm'].values
    mask = usd_ask_cm < usd_per_token_cm
    usd_ask_cm[mask] = usd_per_token_cm[mask] + bidask_cm[mask]/2
    df['usd_ask_cm'] = usd_ask_cm

    df.loc[df.usd_ask_size_cm>1e5, 'usd_ask_size_cm'] = np.nan
    df.loc[df.usd_bid_size_cm>1e5, 'usd_ask_size_cm'] = np.nan
    df['usd_bid_size_cm'].fillna(df.groupby('date')['usd_bid_size_cm'].transform('max'), inplace=True)
    df['usd_ask_size_cm'].fillna(df.groupby('date')['usd_ask_size_cm'].transform('max'), inplace=True)

    df.loc[(df.bidask_cm/df.usd_per_token_cm) > 0.1, 'bidask_cm'] = np.nan
    df['bidask_cm'].fillna(df.groupby('date')['bidask_cm'].transform('median'), inplace=True)

    df.loc[(df.usd_bid_cm/df.usd_per_token_cm) < 0.9, 'usd_bid_cm'] = np.nan
    df.loc[df.usd_bid_cm.isnull(), 'usd_bid_cm'] = (df.loc[df.usd_bid_cm.isnull(), 'usd_per_token_cm']
        -df.loc[df.usd_bid_cm.isnull(), 'bidask_cm']/2)

    df.loc[(df.usd_ask_cm/df.usd_per_token_cm) > 1.1, 'usd_ask_cm'] = np.nan
    df.loc[df.usd_ask_cm.isnull(), 'usd_ask_cm'] = (df.loc[df.usd_ask_cm.isnull(), 'usd_per_token_cm']
        +df.loc[df.usd_ask_cm.isnull(), 'bidask_cm']/2)

    assert 0 == df.bidask_cm.isnull().sum()
    assert 0 == df.usd_bid_cm.isnull().sum()
    assert 0 == df.usd_ask_cm.isnull().sum()
    assert 0 == df.usd_bid_size_cm.isnull().sum()
    assert 0 == df.usd_ask_size_cm.isnull().sum()

    # Create separate dataframes
    btc_cols = ['date',    
        "adr_act_cnt_cm", "adr_act_rec_cnt_cm", "adr_act_sent_cnt_cm",
        "adr_bal_cnt_cm", "cap_act_1yr_usd_cm",
        "cap_mvrv_cur_cm", "cap_mrkt_ffusd_cm",
        "cap_real_usd_cm", "diff_mean_cm",
        "fee_med_usd_cm", "fee_rev_pct_cm", "fee_tot_usd_cm",
        "flow_in_ex_usd_cm", "flow_out_ex_usd_cm",
        "flow_miner_net_0hop_all_usd_cm", "hash_rate_cm",
        "iss_tot_usd_cm", 'mcrc_cm', 'mctc_cm', 'momr_cm', 
        'mri_0hop_all30d_cm', 'ndf_cm', "nvt_adj_ff_cm", 
        "puell_mul_rev_cm", "puell_mul_tot_cm", "rvt_adj_cm",
        "rev_hash_usd_cm", "rev_usd_cm",
        "ser_cm", "sopr_cm", "sply_act_ever_cm",
        "sply_act_10yr_cm", "sply_act_5yr_cm",
        "sply_act_1yr_cm", "sply_act_180d_cm",
        "sply_act_30d_cm", "sply_act_7d_cm",
        "sply_act_1d_cm", "sply_act_pct_1yr_cm",
        "sply_adr_bal_usd_1_cm", "sply_adr_bal_usd_100_cm",
        "sply_adr_bal_usd_10k_cm", "sply_adr_bal_usd_1m_cm",
        "sply_adr_top_100_cm", "sply_adr_top_1pct_cm",
        "sply_ex_usd_cm", "sply_cur_cm", "sply_ff_cm", 
        "sply_exp_fut_10yr_cm", "sply_miner_0hop_all_usd_cm",
        "sply_rvv_5yr_cm", "sply_rvv_1yr_cm", "sply_rvv_180d_cm", 
        "sply_rvv_30d_cm", "sply_rvv_7d_cm", "sply_utxo_loss_cm", 
        "sply_utxo_prof_cm", "tx_tfr_cnt_cm", 
        "tx_tfr_val_adj_usd_cm", "tx_tfr_val_day_dst_cm",
        "tx_tfr_val_med_usd_cm", "tx_tfr_val_usd_cm",
        "utxo_age_med_cm", "utxo_prof_unreal_usd_cm",
        "utxo_loss_unreal_usd_cm", "vel_act_1yr_cm", "vel_cur_1yr_cm"]
    btc_df = df[df.asset_cm=='btc'][btc_cols].reset_index(drop=True)

    eth_cols = ['date', "adr_act_cnt_cm", "adr_act_cont_cnt_cm",
        "adr_act_rec_cnt_cm", "adr_act_sent_cnt_cm",
        "adr_bal_cnt_cm", "cap_act_1yr_usd_cm",
        "cap_mvrv_cur_cm", "cap_mrkt_ffusd_cm",
        "cap_real_usd_cm", "cont_erc_20_cnt_cm",
        "fee_med_usd_cm", "fee_rev_pct_cm",
        "fee_tot_usd_cm", "flow_in_ex_usd_cm",
        "flow_out_ex_usd_cm", "gas_used_tx_cm", "iss_tot_usd_cm",
        "ndf_cm", "nvt_adj_ff_cm", "puell_mul_rev_cm",
        "puell_mul_tot_cm", "rvt_adj_cm", "rev_hash_usd_cm",
        "rev_usd_cm", "ser_cm", "sply_act_ever_cm",
        "sply_act_10yr_cm", "sply_act_5yr_cm",
        "sply_act_1yr_cm", "sply_act_180d_cm",
        "sply_act_30d_cm", "sply_act_7d_cm", "sply_act_1d_cm",
        "sply_act_pct_1yr_cm", "sply_adr_bal_usd_1_cm",
        "sply_adr_bal_usd_100_cm", "sply_adr_bal_usd_10k_cm",
        "sply_adr_bal_usd_1m_cm", "sply_adr_top_100_cm",
        "sply_adr_top_1pct_cm", "sply_burnt_usd_cm",
        "sply_ex_usd_cm", "sply_cur_cm", "sply_ff_cm",
        "sply_exp_fut_10yr_cm", "tx_tfr_cnt_cm",
        "tx_tfr_val_adj_usd_cm", "tx_tfr_val_med_usd_cm",
        "tx_tfr_val_usd_cm", "vel_act_1yr_cm", "vel_cur_1yr_cm"]
    eth_df = df[df.asset_cm=='eth'][eth_cols].reset_index(drop=True)

    xsec_sum_cols = ['adr_act_cnt_cm', 'adr_act_rec_cnt_cm',
        'adr_act_sent_cnt_cm', 'adr_bal_cnt_cm',
        'adr_bal_usd_100_cnt_cm', 'adr_bal_usd_10k_cnt_cm',
        'adr_bal_usd_100k_cnt_cm', 'adr_bal_usd_1m_cnt_cm',
        'cap_act_1yr_usd_cm', 'cap_fut_exp_10yr_usd_cm',
        'cap_mvrv_cur_cm', 'cap_mrkt_cur_usd_cm',
        'cap_mrkt_ffusd_cm', 'cap_real_usd_cm', 'fee_tot_usd_cm',
        'iss_tot_usd_cm', 'rev_usd_cm', 'sply_act_ever_cm',
        'sply_act_10yr_cm', 'sply_act_5yr_cm', 'sply_act_1yr_cm',
        'sply_act_180d_cm', 'sply_act_30d_cm', 'sply_act_7d_cm',
        'sply_act_1d_cm', 'sply_adr_bal_usd_1_cm',
        'sply_adr_bal_usd_100_cm', 'sply_adr_bal_usd_10k_cm',
        'sply_adr_bal_usd_1m_cm', 'sply_adr_top_100_cm',
        'sply_adr_top_1pct_cm', 'sply_ff_cm', 'tx_tfr_cnt_cm',
        'tx_tfr_val_adj_usd_cm', 'tx_tfr_val_usd_cm',
        'usd_ask_size_cm', 'usd_bid_size_cm']
    xsec_sum_df = df.groupby('date')[xsec_sum_cols].sum().reset_index()

    xsec_avg_cols = ['fee_mean_usd_cm', 'fee_med_usd_cm',    
        'fee_rev_pct_cm', 'ndf_cm', 'nvt_adj_ff_cm', 'rvt_adj_cm',    
        'ser_cm', 'sopr_cm', 'sply_act_pct_1yr_cm',
        'vel_act_1yr_cm', 'vel_cur_1yr_cm', 'bidask_cm']
    xsec_avg_df = df.groupby('date')[xsec_avg_cols].mean().reset_index()

    panel_df = df[['date', 'asset_cm', 'cap_mrkt_est_usd_cm', 'reference_rate_usd_cm', 
        'trades_cm', 'usd_per_token_cm', 'usd_volume_cm', 'bidask_cm',
        'usd_ask_cm', 'usd_bid_cm', 'usd_ask_size_cm', 'usd_bid_size_cm']].copy()

    # forwarwd fill missings
    eth_df = eth_df.ffill()
    panel_df.sort_values(by=['asset_cm', 'date'], inplace=True)
    panel_df = panel_df.groupby('asset_cm').apply(lambda group: group.fillna(method='ffill'))

    # Fill remaining missingness with crossectional median besides price, mcap, and volume
    cols_with_missing = [col for col in panel_df.columns if panel_df[col].isna().any()]
    for col in cols_with_missing:
        medians = panel_df.groupby('date')[col].transform('median')
        panel_df[col].fillna(medians, inplace=True)

    # confirm no missing
    for temp_df in [btc_df, eth_df, xsec_sum_df, xsec_avg_df, panel_df]:
        assert 0 == temp_df.isnull().sum().sum()

    # ensure no duplicates by date and asset
    assert not panel_df.duplicated(subset=['date', 'asset_cm']).any()

    # Sort by date then asset and reset index
    panel_df = panel_df.sort_values(by=['date', 'asset_cm'], ignore_index=True)

    # Add btc and eth to col names for those dfs
    btc_df.columns = [col if col == 'date' else 'btc_'+col for col in btc_df.columns]
    eth_df.columns = [col if col == 'date' else 'eth_'+col for col in eth_df.columns]
    xsec_sum_df.columns = [col if col == 'date' else 'total_'+col for col in xsec_sum_df.columns]
    xsec_avg_df.columns = [col if col == 'date' else 'avg_'+col for col in xsec_avg_df.columns]

    # Form temp_macro_df 
    temp_macro_df = btc_df.merge(eth_df, on='date', how='outer', validate='one_to_one')
    temp_macro_df = temp_macro_df.merge(xsec_sum_df, on='date', how='outer', validate='one_to_one')
    temp_macro_df = temp_macro_df.merge(xsec_avg_df, on='date', how='outer', validate='one_to_one')

    return panel_df, temp_macro_df

In [3]:
def cleanMacro(macro_df: pd.DataFrame, temp_macro_df: pd.DataFrame) -> pd.DataFrame:
    """ Clean the CM macro data. """
    # Ensure date is top of hour
    assert 0 == macro_df.date.isnull().sum()
    assert len(macro_df) == macro_df[macro_df.date.dt.minute==0].shape[0]
    macro_df = macro_df.sort_values(by='date', ignore_index=True)

    # Cut down to relevant dates between July 1, 2016 and January 2, 2023
    macro_df = macro_df[(macro_df['date'] >= '2016-07-01') & (macro_df['date'] <= '2023-01-02')]

    # Ensure all dates are present
    macro_df.set_index('date', inplace=True)
    min_dt, max_dt = macro_df.index.min(), macro_df.index.max()
    full_date_range = pd.date_range(start=min_dt, end=max_dt, freq='1H')
    assert len(full_date_range) == macro_df.shape[0]
    macro_df = macro_df.reset_index()

    # Merge together the dataframes
    macro_df = macro_df.merge(temp_macro_df, on='date', how='outer', validate='one_to_one')

    # Rename columns
    macro_df = macro_df.rename(columns={'ex_open_interest_reported_future_usd': 'ex_open_interest_future_usd',
                                        'ex_volume_reported_future_usd_1h': 'ex_volume_future_usd',
                                        'ex_volume_reported_spot_usd_1h': 'ex_volume_spot_usd',
                                        'us_ex_open_interest_reported_future_usd': 'us_ex_open_interest_future_usd',
                                        'us_ex_volume_reported_future_usd_1h': 'us_ex_volume_future_usd',
                                        'us_ex_volume_reported_spot_usd_1h': 'us_ex_volume_spot_usd'})

    # Form stablecoin deviation from one column
    macro_df.loc[macro_df.usd_per_usdc.isnull(), 'usdt_usdc_dev_from_one'] = macro_df.usd_per_usdt - 1
    macro_df.loc[macro_df.usd_per_usdc.notnull()
        & (np.abs(macro_df.usd_per_usdc-1) > np.abs(macro_df.usd_per_usdt-1)), 'usdt_usdc_dev_from_one'] = macro_df.usd_per_usdc-1
    macro_df.loc[macro_df.usd_per_usdc.notnull()
        & (np.abs(macro_df.usd_per_usdc-1) <= np.abs(macro_df.usd_per_usdt-1)), 'usdt_usdc_dev_from_one'] = macro_df.usd_per_usdt-1
    macro_df = macro_df.drop(['usd_per_usdc', 'usd_per_usdt'], axis=1)

    # Confirm no missings in the df
    assert(macro_df.isnull().sum().sum() == 0)

    # Set column order
    cols = list(macro_df.columns.values)
    cols.remove('date')
    cols.sort()
    macro_df = macro_df[['date']+cols]

    # Convert all columns to float32
    for col in cols:
        macro_df[col] = macro_df[col].astype('float32')

    # Ensure all columns have cm in name
    macro_df.columns = [col if col == 'date' or col.endswith('_cm') else col + '_cm' for col in macro_df.columns]

    # ensure no duplicates by date and asset
    assert not macro_df.duplicated(subset=['date']).any()

    # Sort by date then asset and reset index
    macro_df = macro_df.sort_values(by=['date'], ignore_index=True)

    return macro_df

In [4]:
if __name__ == "__main__":
    # set args
    ASSET_IN_FP = '../data/derived/asset_universe_dict.pickle'
    PANEL_IN_FP = '../data/raw/coinmetrics_panel_hourly.pkl'
    MACRO_IN_FP = '../data/raw/coinmetrics_macro_hourly.pkl'
    PANEL_OUT_FP = "../data/derived/cm_panel.pkl"
    MACRO_OUT_FP = '../data/derived/cm_macro.pkl'

    # import
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    macro_df = pd.read_pickle(MACRO_IN_FP)
    df = pd.read_pickle(PANEL_IN_FP)

    # clean
    panel_df, temp_macro_df = cleanPanel(df, asset_universe_dict)
    macro_df = cleanMacro(macro_df, temp_macro_df)

    # output
    panel_df.to_pickle(PANEL_OUT_FP)
    macro_df.to_pickle(MACRO_OUT_FP)


/tmp/ipykernel_247648/3152524850.py:267: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  panel_df = panel_df.groupby('asset_cm').apply(lambda group: group.fillna(method='ffill'))
